In [280]:
from typing import Any
import json
from datetime import datetime
from datetimerange import DateTimeRange
import re

In [328]:
# Le combinazioni valide sono quelle per le quali, PER OGNI CAMPO,
# è presente almeno 1 elemento della current selection

def gen_time_range_from_string(string: str) -> DateTimeRange:
        dates = re.split(";|/", string)
        time_range = DateTimeRange(dates[0], dates[1])
        time_range.start_time_format = "%Y-%m-%d"
        time_range.end_time_format = "%Y-%m-%d"
        if time_range.is_valid_timerange():
            return time_range
        else:
            raise ValueError('Selected start date must be before end date')


def get_possible_values(
    current_selection: dict[str, set[Any]],
    valid_combinations: list[dict[str, set[Any]]],
) -> dict[str, set[Any]]:
    
    result: dict[str, set[Any]] = {}
    for valid_combination in valid_combinations:
        ok = True
        for filed_name, selected_values in current_selection.items():
            if (len(selected_values & valid_combination[filed_name]) == 0):
                ok = False
                break
        if ok:
            for filed_name, valid_values in valid_combination.items():
                current = result.setdefault(filed_name, set())  
                current |= valid_values
    return result    


def load_combinations(path : str) -> list[dict[str, set[Any]]] :
    combinations = json.load(open(path))
    for combination in combinations:
        for field_name, field_values in combination.items():
            combination[field_name] = set(field_values)
    return combinations

def drop_unavailable(
    selected_time_range: set[str],
    valid_combinations: list[dict[str, set[Any]]],
) -> dict[str, set[Any]]:
    
    selected_time_range = gen_time_range_from_string(selected_time_range)
    for i in range(len(valid_combinations)-1, -1, -1):
        ok = False;
        valid_time_ranges = [gen_time_range_from_string(string) for string in valid_combinations[i]["date"]]
        for valid_time_range in valid_time_ranges:
            if selected_time_range in valid_time_range:
                ok = True
                break
        if not ok:
            del valid_combinations[i]
        else:
            valid_combinations[i].pop("date")
    return valid_combinations

In [329]:
def apply_constraints(valid_combinations, selection):
    if ("date" in valid_combinations[0]) & ("date" in selection.keys()):
        valid_combinations = drop_unavailable(next(iter(selection["date"])), valid_combinations)
        date = selection.pop("date") # pop out "date" otherwise get_possible_values will not work
        possible_values = get_possible_values(selection, valid_combinations)
        if possible_values:
            possible_values.setdefault("date", date) # get "date" back in, use selected time range
    else:
        possible_values = get_possible_values(selection, valid_combinations)
    return possible_values

In [330]:
path = "/Users/nicola/devel/notebooks/data/constraints/eac4_constraints.json"
valid_combinations = load_combinations(path)

current_selection: dict[str, set[Any]] = {
        # valid time range: {'2003-01-01/2021-12-31'},
        #'date': {'2040-01-02/2080-01-13'},
        #'date': {'1980-01-02/1990-01-13'},
        #'date': {'2000-01-02/2010-01-13'},
        #'date': {'2010-01-02/2030-01-13'},
        'date': {'2005-01-02/2010-01-13'},
        'variable': {'10m_u_component_of_wind'},
        'time': {'00:00'},
}
       
apply_constraints(valid_combinations, current_selection)

{'time': {'00:00',
  '03:00',
  '06:00',
  '09:00',
  '12:00',
  '15:00',
  '18:00',
  '21:00'},
 'variable': {'10m_u_component_of_wind',
  '10m_v_component_of_wind',
  '2m_dewpoint_temperature',
  '2m_temperature',
  'black_carbon_aerosol_optical_depth_550nm',
  'dust_aerosol_optical_depth_550nm',
  'high_cloud_cover',
  'high_vegetation_cover',
  'lake_cover',
  'land_sea_mask',
  'leaf_area_index_high_vegetation',
  'leaf_area_index_low_vegetation',
  'lifting_threshold_speed',
  'low_cloud_cover',
  'low_vegetation_cover',
  'mean_altitude_of_maximum_injection',
  'mean_sea_level_pressure',
  'medium_cloud_cover',
  'near_ir_albedo_for_diffuse_radiation',
  'near_ir_albedo_for_direct_radiation',
  'organic_matter_aerosol_optical_depth_550nm',
  'particulate_matter_10um',
  'particulate_matter_1um',
  'particulate_matter_2.5um',
  'sea_ice_cover',
  'sea_salt_aerosol_optical_depth_550nm',
  'sea_surface_temperature',
  'skin_reservoir_content',
  'skin_temperature',
  'snow_albedo',

In [331]:
path = "/Users/nicola/devel/notebooks/data/constraints/cmip6_constraints.json"
valid_combinations = load_combinations(path)

current_selection: dict[str, set[Any]] = {
    "experiment" : {"historical"},
    "variable": {"specific_humidity"},
    "year": {"2014", "2000"}
}

apply_constraints(valid_combinations, current_selection)

{'experiment': {'historical'},
 'level': {'1',
  '10',
  '100',
  '1000',
  '150',
  '20',
  '200',
  '250',
  '30',
  '300',
  '400',
  '5',
  '50',
  '500',
  '600',
  '70',
  '700',
  '850',
  '925'},
 'model': {'access_cm2',
  'access_esm1_5',
  'awi_cm_1_1_mr',
  'awi_esm_1_1_lr',
  'bcc_esm1',
  'canesm5',
  'canesm5_canoe',
  'cesm2',
  'cesm2_fv2',
  'cesm2_waccm',
  'cesm2_waccm_fv2',
  'ciesm',
  'cmcc_cm2_hr4',
  'cmcc_cm2_sr5',
  'cmcc_esm2',
  'cnrm_cm6_1',
  'cnrm_cm6_1_hr',
  'cnrm_esm2_1',
  'e3sm_1_0',
  'e3sm_1_1_eca',
  'ec_earth3_aerchem',
  'ec_earth3_cc',
  'ec_earth3_veg_lr',
  'fgoals_f3_l',
  'fgoals_g3',
  'fio_esm_2_0',
  'gfdl_esm4',
  'hadgem3_gc31_ll',
  'hadgem3_gc31_mm',
  'iitm_esm',
  'inm_cm4_8',
  'inm_cm5_0',
  'ipsl_cm5a2_inca',
  'ipsl_cm6a_lr',
  'kace_1_0_g',
  'kiost_esm',
  'mcm_ua_1_0',
  'miroc6',
  'miroc_es2l',
  'mpi_esm1_2_hr',
  'mpi_esm1_2_lr',
  'mri_esm2_0',
  'nesm3',
  'norcpm1',
  'noresm2_mm',
  'sam0_unicon',
  'taiesm1',
  'uke

In [332]:
path = "/Users/nicola/devel/notebooks/data/constraints/gfas_constaints.json"
valid_combinations = load_combinations(path)

current_selection: dict[str, set[Any]] = {
    "variable": {"wildfire_overall_flux_of_burnt_carbon"},
    "date": {"2003-01-01/2022-10-09"}
}
apply_constraints(valid_combinations, current_selection)

{'variable': {'altitude_of_plume_top',
  'mean_altitude_of_maximum_injection',
  'wildfire_combustion_rate',
  'wildfire_flux_of_acetaldehyde',
  'wildfire_flux_of_acetone',
  'wildfire_flux_of_ammonia',
  'wildfire_flux_of_benzene',
  'wildfire_flux_of_black_carbon',
  'wildfire_flux_of_butanes',
  'wildfire_flux_of_butenes',
  'wildfire_flux_of_carbon_dioxide',
  'wildfire_flux_of_carbon_monoxide',
  'wildfire_flux_of_dimethyl_sulfide',
  'wildfire_flux_of_ethane',
  'wildfire_flux_of_ethanol',
  'wildfire_flux_of_ethene',
  'wildfire_flux_of_formaldehyde',
  'wildfire_flux_of_heptane',
  'wildfire_flux_of_hexanes',
  'wildfire_flux_of_hexene',
  'wildfire_flux_of_higher_alkanes',
  'wildfire_flux_of_higher_alkenes',
  'wildfire_flux_of_hydrogen',
  'wildfire_flux_of_isoprene',
  'wildfire_flux_of_methane',
  'wildfire_flux_of_methanol',
  'wildfire_flux_of_nitrogen_oxides',
  'wildfire_flux_of_nitrous_oxide',
  'wildfire_flux_of_non_methane_hydrocarbons',
  'wildfire_flux_of_octene'

In [343]:
# from the form definition
possible_selections: dict[str, list[Any]] = {
    "param": ["T", "Z"],
    "level": ["1000", "850", "500"],
    "step": ["24", "36", "48"],
}
    
current_selection: dict[str, set[Any]] = {
    "param": {"T", "Z"},
    "level": {"1000"},
    "step": {"36"}
}
 
valid_combinations: list[dict[str, set[Any]]] = [
    {"level": {"500"}, "param": {"Z", "T"}, "step": {"24", "36", "48"}},
    {"level": {"1000"}, "param": {"Z"}, "step": {"24", "48"}},
    {"level": {"850"}, "param": {"T"}, "step": {"36", "48"}},
]

apply_constraints(valid_combinations, current_selection)

{}

In [342]:
current_selection: dict[str, set[Any]] = {
    "param": {"T"},
    #"level": {'850'},
}


valid_combinations: list[dict[str, set[Any]]] = [{
    "date": {"1990-01-01;1999-12-31", "2010-10-10;2011-11-11"},
    "city": {"rome", "paris", "london"},
    "level": {"500"},
    "param": {"Z"},
    "step": {"24", "36", "48"}
  }, {
    "date": {"1990-01-01;2011-12-31"},
    "city": {"paris", "london"},
    "level": {"1000"},
    "param": {"Z"},
    "step": {"24", "48"}
  }, {
    "date": {"1980-01-01;2011-12-31"},
    "city": {"rome", "paris", "london"},
    "level": {"850"},
    "param": {"T"},
    "step": {"36", "48"}
  }]

apply_constraints(valid_combinations, current_selection)

{'date': {'1980-01-01;2011-12-31'},
 'city': {'london', 'paris', 'rome'},
 'level': {'850'},
 'param': {'T'},
 'step': {'36', '48'}}